In [1]:
import os
import shutil

from leer_txt import leer_txt 

# Se definen como funciones para que sea mas comodo iterar
# en las celdas de más abajo

def genera_config_txt(txt_deformacion,config_base,save_dir):
    basename=os.path.basename(txt_deformacion)
    save_name=os.path.splitext(basename)[0].replace('.','')
    
    #direccion completa de guardado (incl. nombre de archivo)
    config_txt=os.path.join(save_dir,'config_'+save_name+'.txt')
    
    lineas_base=leer_txt(config_base)
    lineas_deformacion=leer_txt(txt_deformacion)
    
    n_faults=len(lineas_deformacion) #Numero de fallas 
    linea_nfallas=4 # Linea del archivo config con cantidad de fallas
    
    # Agrega cantidad de fallas a la linea correspondiente
    lineas_base[linea_nfallas]=str(n_faults)+lineas_base[linea_nfallas]
    
    # Agrega lineas de deformacion
    linea_base_def=5 # Linea en donde comienzan a anotarse las deformaciones
    del(lineas_base[linea_base_def])
    i=0
    for line in lineas_deformacion:
        
        line=line.split(' ')
        line[1]=str(float(line[1])+360)
        line=' '.join(line)
        
        lineas_base.insert(i+linea_base_def,line)
        i=i+1
    
    # Escribe el archivo .txt
    with open(config_txt, 'w', newline='\n') as writer:
        for linea in lineas_base:
            writer.write(linea+'\n')

    print(config_txt) # muestra la ubicacion del archivo creado
    return config_txt
    
def genera_sh(sh_base,config,save_dir):
    
    # Mismo nombre que archivo de config, solo que cambia "config_" por "run_"
    config_name=os.path.basename(config) #incluye ".txt"
    escenario=(os.path.splitext(config_name)[0]).split('_')[1] # quita el ".txt"
    savename='run'
    
    #direccion completa de guardado
    sh_nuevo=os.path.join(save_dir,savename+'.sh')
    
    lineas_sh=leer_txt(sh_base)
    
    lineas_sh.insert(2,'#SBATCH -J '+escenario)
    
    
    #borra las dos ultimas lineas y agrega nombre de archivo config
    del(lineas_sh[-2:])
    lineas_finales=['get_load_balancing-1\t\t'+config_name+' 1 1',
                    'mpiexec TsunamiHySEA-gpuk-1\t'+config_name]
    lineas_sh.extend(lineas_finales)
    
    with open(sh_nuevo, 'w', newline='\n') as writer:
        for linea in lineas_sh:
            writer.write(linea+'\n')
    
    print(sh_nuevo)
    return sh_nuevo

In [14]:
# Inputs

sh_base=r"C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Bases\run_base.sh"
config_base=r"C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Bases\config_base.txt"
deformaciones_dir=r"C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8"
ts_file=r"C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\ts_config1.xy"

save_dir=r"C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8"

In [15]:
# recorre una carpeta con deformaciones,
# crea una carpeta por cada informacion y
# genera config.txt, run.sh y agrega archivo.ts

with os.scandir(deformaciones_dir) as entries:
    for entry in entries:
        if entry.is_file():
            if entry.name.endswith(".txt"):
                folder_name=os.path.splitext(entry.name)[0].replace('.','')
                save_dir_def=os.path.join(save_dir,folder_name)
                os.mkdir(save_dir_def) #crea carpeta en nueva direccion
    
                txt_deformacion=os.path.abspath(entry)
                
                print(save_dir_def)
                config_txt=genera_config_txt(txt_deformacion,config_base,save_dir_def)
                genera_sh(sh_base,config_txt,save_dir_def)
                shutil.copy(ts_file, save_dir_def)
                print()

C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8\z2ap000100
C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8\z2ap000100\config_z2ap000100.txt
C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8\z2ap000100\run.sh

C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8\z2ap000101
C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8\z2ap000101\config_z2ap000101.txt
C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8\z2ap000101\run.sh

C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8\z2ap000102
C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\mw_8.8\z2ap000102\config_z2ap000102.txt
C:\Users\ASUS\OneDrive - Universidad Técnica Federico Santa María\Memoria\Dependencia\